# Backends - t|ket> example

This example shows how to use pytket to export quantum circuits to the various allowed backends. As t|ket> is designed to be platform agnostic, any t|ket> circuit can be run using any instance of the `Backend` class for maximum portability. The circuit will be optimised, rebased and routed for the appropriate backend automatically - although this feature can be switched off. We currently provide an interface to a variety of simulators and any IBM quantum experience chip that the user has access to, and this interface uses a common structure so it is easy to switch between backends. The full list at the time of writing is:

* ProjectQ simulator
* Aer state-vector simulator
* Aer QASM simulator
* IBMQ devices

This list will soon expand.
Note that the qasm simulator can take in a noise model, and an API for this is provided through pytket as well.

To get started, we must install core pytket and the subpackages required to interface with the two providers:

```
pip install pytket
pip install pytket_qiskit
pip install pytket_projectq
```

First, import the backends that we will be demonstrating.

In [1]:
from pytket.backends.ibm import AerStateBackend, AerBackend, IBMQBackend
from pytket.backends.projectq import ProjectQBackend

We are also going to be making a circuit to run on these backends, so import the `Circuit` class.

In [2]:
from pytket import Circuit

Below we generate a circuit which will produce a Bell state, assuming the qubits are all initialised in the |0> state:

In [3]:
circ = Circuit(2,2)
circ.H(0)
circ.CX(0,1)

<tket::Circuit qubits=4, gates=2>

The first demo will be of the `AerStateBackend` - we are going to verify that circuit `circ` does produce 2 qubits in the Bell state. Create an `AerStateBackend` object and then call `get_state` on that object, using the circuit as an argument, to retrieve the results. This style of usage is used consistently in the pytket backends.

In [4]:
aer_state_b = AerStateBackend()
statevector = aer_state_b.get_state(circ)
print(statevector)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


This statevector is a numpy array. If we call the output statevector $|\psi_{circ}>$, clearly $|\psi_{circ}> = \frac{(|00> + |11>)}{\sqrt2}$.

Now we want to measure this statevector to get some actual results out, so let's append some `Measure` gates to the circuit. We can automatically add `Measure` gates to the end of every qubit using the `measure_all()` method.

In [5]:
circ.measure_all()

<tket::Circuit qubits=4, gates=4>

We can get some shots out from the `AerBackend`, which is an interface to the Qiskit QASM simulator. We would like to get 10 shots out this time, and we can seed the simulator to guarantee we get the same shot count each time we run the same circuit:

In [6]:
aer_b = AerBackend()
shots = aer_b.run(circuit=circ,shots=10,seed=1)
print(shots)

[[1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]]


As you can see, there is a 40/60 split between "00" results and "11" results. If we change the seed, or remove it, we will get varying results according to the pseudo-random number generation internal to Qiskit's QASM simulator. What happens if we simulate some noise in our imagined device, using Qiskit Aer's noise model?

To demonstrate this, we will require an import from Qiskit. For more information about noise modeling using Qiskit Aer, see the [Qiskit device noise simulation docs](https://qiskit.org/documentation/aer/device_noise_simulation.html).

In [7]:
from qiskit.providers.aer.noise import NoiseModel
my_noise_model = NoiseModel()
readout_error = 0.2
my_noise_model.add_all_qubit_readout_error([[1-readout_error,readout_error],[readout_error,1-readout_error]])

This simple noise model gives a 20% chance that, upon measurement, a qubit which would otherwise have been measured as "0" would instead be measured as "1", and vice versa. Let's see what our shot table looks like with this model!

In [8]:
noisy_aer_b = AerBackend(my_noise_model)
noisy_shots = noisy_aer_b.run(circuit=circ,shots=10,seed=1)
print(noisy_shots)

[[1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [0 1]]


We now have some spooky "01" and "10" measurements, which could never happen when measuring a Bell state on a noiseless device. 

The `AerBackend` class can accept any Qiskit noise model.

The last simulator we will demo is the `ProjectQBackend`. We can use the `ProjectQBackend` to get fast expectation values from circuits in conjunction with ProjectQ's `QubitOperator` class. Note that it is possible to use the `AerStateBackend` for this purpose as well, but ProjectQ is designed to retrieve expectation values very quickly. This is useful for the noiseless simulation of many quantum algorithms.

Note: ProjectQ can also produce shots in the style of `AerBackend`, using the same method, but cannot accept a noise model.

For an explanation of ProjectQ's `QubitOperator` class, see the [docs](https://projectq.readthedocs.io/en/latest/projectq.ops.html#projectq.ops.QubitOperator).

Let's create a `QubitOperator` object and a new circuit:

In [9]:
import projectq
from projectq.ops import QubitOperator

hamiltonian = 0.5 * QubitOperator('X0 X2') + 0.3 * QubitOperator('Z0')

In [10]:
circ2 = Circuit(3)
circ2.Y(0)
circ2.H(1)
circ2.Rx(2,0.3)

<tket::Circuit qubits=3, gates=3>

Now we can create a `ProjectQBackend` instance and feed it our circuit and `QubitOperator`:

In [11]:
projectq_b = ProjectQBackend()
expectation = projectq_b.get_operator_expectation_value(circ2,hamiltonian)
print(expectation)

-0.2999999999999999


The last leg of this tour includes running a pytket circuit on an actual quantum computer. To do this, you will need an IBM quantum experience account, which you can get for free here: https://quantumexperience.ng.bluemix.

You can see the device specs on the Qiskit [GitHub repo](https://github.com/Qiskit/ibmq-device-information).

Let's take our circuit from before and run it using the `IBMQBackend` on the "ibmqx4" device.

In [12]:
ibmq_b = IBMQBackend("ibmqx4")
quantum_shots = ibmq_b.run(circuit=circ,shots=10)
print(quantum_shots)

Job Status: job has successfully run
[[0 1]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [1 1]
 [0 0]]


These are from an actual device, so it's impossible to perfectly predict what the results will be. However, due to the problem of noise, it would be unsurprising if there are a few "01" or "10" results in the table. The circuit is very short, so it should be fairly close to the ideal result.

This concludes our tour of the pytket backends. For more information, read our [docs](https://pytket.readthedocs.io) or see the other examples on our [GitHub repo](https://github.com/CQCL/pytket).